In [1]:
library(data.tree)

In [11]:
top <- Node$new("Advice")
top$input <- "D_C, C"
l1 <- top$AddChild("Leaf")
l1$input <- "D_C, C"
e1 <- top$AddChild("Exp. Mech")
c1 <- e1$AddChild("C1")
c2 <- e1$AddChild("C2")
cn <- e1$AddChild("CN")
plot(top)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.
